### ### final3_version3 - 처리 사항
- matchType 통합

- 변수제거
> groupId,matchId,killplace,(killpoint, rankpoint, winpoint => Weighted), numgroups, maxplace

- 이상치 제거
> matchType은 이상치 없음<br>
> 'assists_mean', 'boosts_mean','headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean','longestKill_mean', 'rideDistance_mean','swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean' => quantile 상위 0.005퍼센트 이상치 제거<br>
> 평균값이기 때문에 total distance==0 제거(핵쟁이일 가능성)

- 데이터 변환
> distance 변수 sum
> matchId, groupId 별로 평균값으로 컬럼 대체<br>
> match 평균 변수 추가<br>
> matchsize 변수 추가<br>
> 각_mean 변수 / 각_match_mean 변수 => 컬럼개수 줄이고, 해당 판에서 그 사람의 위치를 비율로 만들어줄 수 있음

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing
import gc

In [11]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 100)
pd.set_option('display.width', None)

In [6]:
df_train = pd.read_csv('./data/train_V2.csv', dtype={
                    'assists': 'int16',
                    'boosts': 'int16',
                    'DBNOs': 'int16',
                    'headshotKills': 'int16',
                    'heals': 'int16',
                    'kills': 'int16',
                    'killStreaks': 'int16',
                    'longestKill': 'float32',
                    'revives': 'int16',
                    'rideDistance': 'float32',
                    'swimDistance': 'float32',
                    'vehicleDestroys': 'int16',
                    'walkDistance': 'float32',
                    'weaponsAquired': 'int16',
                    'winPlacePerc': 'float32'
                })

In [7]:
df_train = df_train[df_train['maxPlace'] > 1]

In [ ]:
# 필요 없는 칼럼 제거
del_list = ['maxPlace','killPlace', 'killPoints',
            'numGroups', 'rankPoints','winPoints','matchDuration']
for col in del_list :
    del df_train[col]

In [9]:
features = list(df_train.columns)    
    
df_train_mean = df_train.groupby(['matchId','groupId']).mean()
df_train_size = df_train.groupby(['matchId']).size().reset_index(name='match_size')
df_train_match_mean = df_train.groupby(['matchId']).mean()
df_train_max = df_train.groupby(['matchId','groupId'])[features].agg('max')
df_train_min = df_train.groupby(['matchId','groupId'])[features].agg('min')

df_train = pd.merge(df_train, df_train_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
del df_train_max
df_train = pd.merge(df_train, df_train_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
del df_train_min
df_train = pd.merge(df_train, df_train_mean, suffixes=["","_mean"], how = 'left', on=['matchId','groupId'])
del df_train_mean
df_train = pd.merge(df_train,df_train_match_mean, suffixes=["","_match_mean"], how="left", on=['matchId'])
del df_train_match_mean
df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId'])
del df_train_size

In [12]:
df_train.head(10)

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,kills,killStreaks,longestKill,matchType,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,DBNOs_max,Id_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,matchType_max,revives_max,rideDistance_max,roadKills_max,swimDistance_max,teamKills_max,vehicleDestroys_max,walkDistance_max,weaponsAcquired_max,winPlacePerc_max,DBNOs_min,Id_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,matchType_min,revives_min,rideDistance_min,roadKills_min,swimDistance_min,teamKills_min,vehicleDestroys_min,walkDistance_min,weaponsAcquired_min,winPlacePerc_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,rideDistance_mean,roadKills_mean,swimDistance_mean,teamKills_mean,vehicleDestroys_mean,walkDistance_mean,weaponsAcquired_mean,winPlacePerc_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,rideDistance_match_mean,roadKills_match_mean,swimDistance_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,walkDistance_match_mean,weaponsAcquired_match_mean,winPlacePerc_match_mean,match_size
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.000,0,0,0,0,0,0.000000,squad-fpp,0,0.0000,0,0.00,0,0,244.800003,1,0.4444,2,c56d45be16aa86,0,0,318.000,1,0,1,4,27.660000,squad-fpp,0,0.000000,0,0.0000,0,0,342.799988,2,0.4444,0,100eef17c4d773,0,0,0.000,0,0,0,0,0.000000,squad-fpp,0,0.0000,0,0.0,0,0,48.279999,1,0.4444,0.0,0.0,102.187500,0.500000,0.250000,0.000000,1.000000,0.250000,6.915000,0.000000,0.000000,0.0,0.000000,0.0,0.0,182.989990,1.250000,0.4444,0.291667,0.718750,113.036979,0.812500,0.218750,0.562500,0.875000,0.520833,13.189053,0.145833,19.144062,0.0,0.007685,0.041667,0.000000,998.698120,3.010417,0.468364,96
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.470,0,0,0,0,0,0.000000,squad-fpp,0,0.0045,0,11.04,0,0,1434.000000,5,0.6400,2,fb37a556eb6a3d,2,3,300.800,0,6,2,2,77.970001,squad-fpp,0,5418.000000,0,11.0400,0,0,2435.000000,8,0.6400,0,3dcf4259b62f66,0,0,0.000,0,0,0,0,0.000000,squad-fpp,0,0.0045,0,0.0,0,0,1434.000000,5,0.6400,0.5,1.5,142.817500,0.500000,0.000000,3.250000,1.000000,0.750000,20.912251,0.000000,3197.001221,0.0,2.760000,0.0,0.0,1892.750000,6.000000,0.6400,0.384615,1.186813,133.582857,0.868132,0.296703,1.384615,0.912088,0.615385,28.534889,0.186813,759.818054,0.0,0.996429,0.010989,0.010989,1572.214233,4.043956,0.456703,91
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.000,0,0,0,0,0,0.000000,duo,0,0.0000,0,0.00,0,0,161.800003,2,0.7755,1,3d588ea15ea8ba,1,3,146.600,0,2,1,2,10.840000,duo,0,340.399994,0,0.0000,0,0,1119.000000,2,0.7755,0,1eaf90ac73de72,0,0,68.000,0,0,0,0,0.000000,duo,0,0.0000,0,0.0,0,0,161.800003,2,0.7755,0.5,1.5,107.300000,0.500000,0.000000,1.000000,1.000000,0.500000,5.420000,0.000000,170.199997,0.0,0.000000,0.0,0.0,640.400024,2.000000,0.7755,0.204082,0.887755,114.429490,0.530612,0.183673,0.877551,0.928571,0.540816,16.685316,0.071429,85.594398,0.0,1.294867,0.000000,0.000000,951.576721,3.173469,0.456686,98
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.900,0,0,0,0,0,0.000000,squad-fpp,0,0.0000,0,0.00,0,0,202.699997,3,0.1667,0,4616d365dd2853,0,0,32.900,0,0,0,0,0.000000,squad-fpp,0,0.000000,0,0.0000,0,0,202.699997,3,0.1667,0,4616d365dd2853,0,0,32.900,0,0,0,0,0.000000,squad-fpp,0,0.0000,0,0.0,0,0,202.699997,3,0.1667,0.0,0.0,32.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,202.699997,3.000000,0.1667,0.296703,1.494505,149.946044,0.857143,0.175824,1.593407,0.901099,0.527473,18.094769,0.263736,71.163734,0.0,0.456703,0.032967,0.000000,1286.025879,3.637363,0.495240,91
4,315c

In [20]:
df_train_cvt = df_train.copy()

In [21]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 3, 'duo': 2, 'solo-fpp': 1, 'squad': 3, 'duo-fpp': 2, 'solo': 1,
       'normal-squad-fpp': 4, 'crashfpp': 4, 'flaretpp': 4, 'normal-solo-fpp': 4,
       'flarefpp': 4, 'normal-duo-fpp': 4, 'normal-duo': 4, 'normal-squad': 4,
       'crashtpp': 4, 'normal-solo': 4 }

df_train_cvt['matchType'] = df_train_cvt['matchType'].replace(matchTyp)

#필요 없는 칼럼 제거
del_list = ['assists', 'boosts','damageDealt', 'DBNOs', 'teamKills','roadKills','vehicleDestroys',
       'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
       'revives', 'rideDistance', 'swimDistance', 'walkDistance',
       'weaponsAcquired','winPlacePerc_match_mean','winPlacePerc_mean','winPlacePerc_max','winPlacePerc_min','matchType_max','matchType_min','Id_max','Id_min',]

for col in del_list :
    del df_train_cvt[col]
    
# # null값 확인 및 
# # inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# # na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
# df_train_cvt.dropna(inplace = True)
# print(df_train_cvt.isnull().any().any())


In [22]:
data = df_train_cvt.copy()

In [23]:
data.columns

Index(['Id', 'groupId', 'matchId', 'matchType', 'winPlacePerc', 'DBNOs_max',
       'assists_max', 'boosts_max', 'damageDealt_max', 'headshotKills_max',
       'heals_max', 'killStreaks_max', 'kills_max', 'longestKill_max',
       'revives_max', 'rideDistance_max', 'roadKills_max', 'swimDistance_max',
       'teamKills_max', 'vehicleDestroys_max', 'walkDistance_max',
       'weaponsAcquired_max', 'DBNOs_min', 'assists_min', 'boosts_min',
       'damageDealt_min', 'headshotKills_min', 'heals_min', 'killStreaks_min',
       'kills_min', 'longestKill_min', 'revives_min', 'rideDistance_min',
       'roadKills_min', 'swimDistance_min', 'teamKills_min',
       'vehicleDestroys_min', 'walkDistance_min', 'weaponsAcquired_min',
       'assists_mean', 'boosts_mean', 'damageDealt_mean', 'DBNOs_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'revives_mean', 'rideDistance_mean',
       'roadKills_mean', 'swimDistance_mean', 'teamKills_m

In [24]:
data['total_distance_max'] = data['rideDistance_max']+data['swimDistance_max']+data['walkDistance_max']

In [25]:
data['total_distance_min'] = data['rideDistance_min']+data['swimDistance_min']+data['walkDistance_min']

In [26]:
data['total_distance_mean'] = data['rideDistance_mean']+data['swimDistance_mean']+data['walkDistance_mean']

In [27]:
data['total_distance_match_mean'] = data['rideDistance_match_mean']+data['swimDistance_match_mean']+data['walkDistance_match_mean']

In [28]:
del data['rideDistance_min']
del data['swimDistance_min']
del data['walkDistance_min']
del data['rideDistance_max']
del data['swimDistance_max']
del data['walkDistance_max']
del data['rideDistance_mean']
del data['swimDistance_mean']
del data['walkDistance_mean']
del data['rideDistance_match_mean']
del data['swimDistance_match_mean']
del data['walkDistance_match_mean']

In [29]:
data.head()

,Id,groupId,matchId,matchType,winPlacePerc,DBNOs_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,revives_max,roadKills_max,teamKills_max,vehicleDestroys_max,weaponsAcquired_max,DBNOs_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,revives_min,roadKills_min,teamKills_min,vehicleDestroys_min,weaponsAcquired_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,roadKills_mean,teamKills_mean,vehicleDestroys_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,roadKills_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,weaponsAcquired_match_mean,match_size,total_distance_max,total_distance_min,total_distance_mean,total_distance_match_mean
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,3,0.4444,2,0,0,318.0,1,0,1,4,27.660000,0,0,0,0,2,0,0,0,0.0,0,0,0,0,0.000000,0,0,0,0,1,0.0,0.0,102.1875,0.5,0.25,0.00,1.0,0.25,6.915000,0.0,0.0,0.0,0.0,1.25,0.291667,0.718750,113.036979,0.812500,0.218750,0.562500,0.875000,0.520833,13.189053,0.145833,0.0,0.041667,0.000000,3.010417,96,342.799988,48.279999,182.989990,1017.849854
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,3,0.6400,2,2,3,300.8,0,6,2,2,77.970001,0,0,0,0,8,0,0,0,0.0,0,0,0,0,0.000000,0,0,0,0,5,0.5,1.5,142.8175,0.5,0.00,3.25,1.0,0.75,20.912251,0.0,0.0,0.0,0.0,6.00,0.384615,1.186813,133.582857,0.868132,0.296703,1.384615,0.912088,0.615385,28.534889,0.186813,0.0,0.010989,0.010989,4.043956,91,7864.040039,1434.004517,5092.511230,2333.028809
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,2,0.7755,1,1,3,146.6,0,2,1,2,10.840000,0,0,0,0,2,0,0,0,68.0,0,0,0,0,0.000000,0,0,0,0,2,0.5,1.5,107.3000,0.5,0.00,1.00,1.0,0.50,5.420000,0.0,0.0,0.0,0.0,2.00,0.204082,0.887755,114.429490,0.530612,0.183673,0.877551,0.928571,0.540816,16.685316,0.071429,0.0,0.000000,0.000000,3.173469,98,1459.400024,161.800003,810.600037,1038.465942
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,3,0.1667,0,0,0,32.9,0,0,0,0,0.000000,0,0,0,0,3,0,0,0,32.9,0,0,0,0,0.000000,0,0,0,0,3,0.0,0.0,32.9000,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,3.00,0.296703,1.494505,149.946044,0.857143,0.175824,1.593407,0.901099,0.527473,18.094769,0.263736,0.0,0.032967,0.000000,3.637363,91,202.699997,202.699997,202.699997,1357.646362
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,1,0.1875,0,0,0,100.0,0,0,1,1,58.529999,0,0,0,0,2,0,0,0,100.0,0,0,1,1,58.529999,0,0,0,0,2,0.0,0.0,100.0000,0.0,0.00,0.00,1.0,1.00,58.529999,0.0,0.0,0.0,0.0,2.00,0.051546,1.175258,118.159175,0.000000,0.309278,1.000000,0.938144,0.494845,22.575207,0.000000,0.0,0.020619,0.000000,3.381443,97,49.750000,49.750000,49.750000,965.124329


In [30]:
#  data 리스트
quantile_list = ['assists_mean', 'boosts_mean', 'damageDealt_mean', 'DBNOs_mean','revives_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'total_distance_mean', 'weaponsAcquired_mean','teamKills_mean']

quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(data[col],0.997),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

{'assists_mean': 2.66667, 'boosts_mean': 8.0, 'damageDealt_mean': 838.8, 'DBNOs_mean': 4.66667, 'revives_mean': 1.66667, 'headshotKills_mean': 3.0, 'heals_mean': 14.0, 'kills_mean': 7.5, 'killStreaks_mean': 2.2, 'longestKill_mean': 250.76666, 'total_distance_mean': 10755.33301, 'weaponsAcquired_mean': 11.0, 'teamKills_mean': 1.0}


In [31]:
# 이상치 제거 1
for col in quantile_dic:
    data = data[data[col] <= quantile_dic[col]]

In [32]:
# 이상치 제거 2
data.drop(index=data[(data['total_distance_max']==0)].index,inplace=True)
data.drop(index=data[(data['total_distance_mean']==0)].index,inplace=True)
data.drop(index=data[(data['total_distance_match_mean']==0)].index,inplace=True)

In [26]:
# # data 변환 - ratio
# mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
# 'total_distance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean', 'roadKills_mean','vehicleDestroys_mean',
#             'damageDealt_mean','teamKills_mean']
# mean_match_list = ['assists_match_mean', 'boosts_match_mean', 'headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 
#                    'killStreaks_match_mean', 'longestKill_match_mean',
#  'total_distance_match_mean', 'weaponsAcquired_match_mean','DBNOs_match_mean','revives_match_mean',
#                    'roadKills_match_mean','vehicleDestroys_match_mean','damageDealt_match_mean','teamKills_match_mean']
# name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
#  'total_distance', 'weaponsAcquired','DBNOs','revives','roadKills','vehicleDestroys','damageDealt','teamKills']
# print(len(mean_list))
# print(len(mean_match_list))
# print(len(name_list))

# for i in range(len(mean_list)):
#     mean = mean_list[i]
#     match = mean_match_list[i]
#     name = name_list[i]  
#     data[name+'_ratio'] = data[mean]/(data[match])
#     data[name+'_ratio'].fillna(value=0,inplace=True)
#     del data[mean]
#     del data[match]

15
15
15


In [33]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# winPlacePerc 컬럼위치 변경
data = change_column_order(data,'winPlacePerc',len(list(data.columns)))

In [34]:
data.describe()

,matchType,DBNOs_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,revives_max,roadKills_max,teamKills_max,vehicleDestroys_max,weaponsAcquired_max,DBNOs_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,revives_min,roadKills_min,teamKills_min,vehicleDestroys_min,weaponsAcquired_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,roadKills_mean,teamKills_mean,vehicleDestroys_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,roadKills_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,weaponsAcquired_match_mean,match_size,total_distance_max,total_distance_min,total_distance_mean,total_distance_match_mean,winPlacePerc
count,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06,4.323383e+06
mean,2.396381e+00,1.348910e+00,4.952652e-01,1.640543e+00,2.133466e+02,4.742231e-01,2.331168e+00,9.589183e-01,1.614145e+00,4.212800e+01,4.032074e-01,7.928976e-03,7.163673e-02,2.131502e-02,4.726251e+00,1.357870e-01,4.411360e-02,5.662057e-01,5.492967e+01,5.492736e-02,5.276565e-01,2.186105e-01,3.302536e-01,7.113585e+00,1.490592e-02,9.187250e-04,3.480608e-03,8.504914e-04,2.598716e+00,2.177219e-01,1.047634e+00,1.233617e+02,6.291712e-01,2.060287e-01,1.277805e+00,8.560983e-01,5.277053e-01,2.113401e+01,1.562711e-01,3.165114e-03,2.334075e-02,7.295907e-03,3.596571e+00,2.307823e-01,1.107623e+00,1.295360e+02,6.588089e-01,2.237846e-01,1.365677e+00,9.134145e-01,5.430081e-01,2.287896e+01,1.652525e-01,3.465196e-03,2.374410e-02,7.890286e-03,3.629941e+00,9.435768e+01,2.085887e+03,1.329811e+03,1.698767e+03,1.761285e+03,4.665227e-01
std,7.515422e-01,1.435604e+00,7.785196e-01,1.962197e+00,1.878217e+02,7.550771e-01,3.329656e+00,8.229507e-01,1.773938e+00,6.523051e+01,6.791728e-01,1.071078e-01,2.905059e-01,1.508876e-01,2.507142e+00,4.203087e-01,2.177921e-01,1.159118e+00,9.351396e+01,2.666241e-01,1.363511e+00,4.424061e-01,7.935797e-01,2.299279e+01,1.211765e-01,3.402897e-02,5.889392e-02,3.005711e-02,1.842259e+00,3.912982e-01,1.419992e+00,1.171398e+02,7.207779e-01,3.809759e-01,1.986360e+00,1.049258e+00,5.010073e-01,3.505563e+01,2.870445e-01,4.998585e-02,1.068458e-01,5.827023e-02,1.927751e+00,1.014282e-01,2.262001e-01,1.343989e+01,3.109800e-01,5.471213e-02,5.002168e-01,9.177726e-02,5.985801e-02,6.255129e+00,9.184739e-02,1.475927e-02,2.359284e-02,1.250044e-02,5.102707e-01,5.861331e+00,2.412179e+03,1.788756e+03,2.007634e+03,8.518691e+02,3.035249e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0

In [35]:
data.head(15)

,Id,groupId,matchId,matchType,DBNOs_max,assists_max,boosts_max,damageDealt_max,headshotKills_max,heals_max,killStreaks_max,kills_max,longestKill_max,revives_max,roadKills_max,teamKills_max,vehicleDestroys_max,weaponsAcquired_max,DBNOs_min,assists_min,boosts_min,damageDealt_min,headshotKills_min,heals_min,killStreaks_min,kills_min,longestKill_min,revives_min,roadKills_min,teamKills_min,vehicleDestroys_min,weaponsAcquired_min,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,revives_mean,roadKills_mean,teamKills_mean,vehicleDestroys_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,damageDealt_match_mean,DBNOs_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,revives_match_mean,roadKills_match_mean,teamKills_match_mean,vehicleDestroys_match_mean,weaponsAcquired_match_mean,match_size,total_distance_max,total_distance_min,total_distance_mean,total_distance_match_mean,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,3,2,0,0,318.000,1,0,1,4,27.660000,0,0,0,0,2,0,0,0,0.000,0,0,0,0,0.000000,0,0,0,0,1,0.000000,0.000000,102.187500,0.500000,0.250000,0.000000,1.000000,0.250000,6.915000,0.000000,0.0,0.0,0.0,1.250000,0.291667,0.718750,113.036979,0.812500,0.218750,0.562500,0.875000,0.520833,13.189053,0.145833,0.000000,0.041667,0.000000,3.010417,96,342.799988,48.279999,182.989990,1017.849854,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,3,2,2,3,300.800,0,6,2,2,77.970001,0,0,0,0,8,0,0,0,0.000,0,0,0,0,0.000000,0,0,0,0,5,0.500000,1.500000,142.817500,0.500000,0.000000,3.250000,1.000000,0.750000,20.912251,0.000000,0.0,0.0,0.0,6.000000,0.384615,1.186813,133.582857,0.868132,0.296703,1.384615,0.912088,0.615385,28.534889,0.186813,0.000000,0.010989,0.010989,4.043956,91,7864.040039,1434.004517,5092.511230,2333.028809,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,2,1,1,3,146.600,0,2,1,2,10.840000,0,0,0,0,2,0,0,0,68.000,0,0,0,0,0.000000,0,0,0,0,2,0.500000,1.500000,107.300000,0.500000,0.000000,1.000000,1.000000,0.500000,5.420000,0.000000,0.0,0.0,0.0,2.000000,0.204082,0.887755,114.429490,0.530612,0.183673,0.877551,0.928571,0.540816,16.685316,0.071429,0.000000,0.000000,0.000000,3.173469,98,1459.400024,161.800003,810.600037,1038.465942,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,3,0,0,0,32.900,0,0,0,0,0.000000,0,0,0,0,3,0,0,0,32.900,0,0,0,0,0.000000,0,0,0,0,3,0.000000,0.000000,32.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.000000,0.296703,1.494505,149.946044,0.857143,0.175824,1.593407,0.901099,0.527473,18.094769,0.263736,0.000000,0.032967,0.000000,3.637363,91,202.699997,202.699997,202.699997,1357.646362,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,1,0,0,0,100.000,0,0,1,1,58.529999,0,0,0,0,2,0,0,0,100.000,0,0,1,1,58.529999,0,0,0,0,2,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,1.000000,1.000000,58.529999,0.000000,0.0,0.0,0.0,2.000000,0.051546,1.175258,118.159175,0.000000,0.309278,1.000000,0.938144,0.494845,22.575207,0.000000,0.000000,0.020619,0.000000,3.381443,97,49.750000,49.750000,49.750000,965.124329,0.1875
5,ff79c12f326506,289a6836a88d27,bac52627a12114,3,3,1,1,183.300,1,0,2,3,18.440001,0,0,0,0,4,0,0,0,0.000,0,0,0,0,0.000000,0,0,0,0,0,0.100000,0.100000,46.974000,0.400000,0.100000,0.000000,0.400000,0.300000,2.991000,0.000000,0.0,0.0,0.0,1.000000,0.210526,1.168421,133.569158,0.863158,0.273684,0.915789,0.947368,0.557895,20.187243,0.231579,0.000000,0.000000,0.000000,3.010526,95,123.300003,0.000000,50.581299,1121.109009,0.0370
6,95959be0e21ca3,2c485a1ad3d0f1,a8274e903927a2,3,2,0,0,200.000,1,0,1,2,15.290000,0,0,0,0,1,0,0,0,0.000,0,0,0,0,0.000000,0,0,0,0,1,0.000000,0.000000,45.333333,0.333333,0.166667,0.000000,0.333333,0.166667,2.548333,0.000000,0.0,0.0,0.0,1.000000,0.237113,0.958763,126.382474,0.814433,0.175258,1.391753,0.886598,0.536082,25.381678,0.175258,0.000000,0.020619,0.000000,2.948454,97,52.349998,12.230000

In [36]:
data = round(data,4)

In [37]:
data.to_csv('./data/final3/final3_Data(version3).csv',sep=',', encoding='utf-8', index=False)

In [38]:
data.columns

Index(['Id', 'groupId', 'matchId', 'matchType', 'DBNOs_max', 'assists_max',
       'boosts_max', 'damageDealt_max', 'headshotKills_max', 'heals_max',
       'killStreaks_max', 'kills_max', 'longestKill_max', 'revives_max',
       'roadKills_max', 'teamKills_max', 'vehicleDestroys_max',
       'weaponsAcquired_max', 'DBNOs_min', 'assists_min', 'boosts_min',
       'damageDealt_min', 'headshotKills_min', 'heals_min', 'killStreaks_min',
       'kills_min', 'longestKill_min', 'revives_min', 'roadKills_min',
       'teamKills_min', 'vehicleDestroys_min', 'weaponsAcquired_min',
       'assists_mean', 'boosts_mean', 'damageDealt_mean', 'DBNOs_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'revives_mean', 'roadKills_mean', 'teamKills_mean',
       'vehicleDestroys_mean', 'weaponsAcquired_mean', 'assists_match_mean',
       'boosts_match_mean', 'damageDealt_match_mean', 'DBNOs_match_mean',
       'headshotKills_match_mean', 'heals_